In [0]:
import os
from torch.utils.data import Dataset
import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import time
import numpy as np
import pandas as pd
import os
import datetime as dt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models

from fastai.vision import *
import fastai
from PIL import Image
import shutil
# import fastai
# from fastai import vision

!pip install pretrainedmodels
import pretrainedmodels as pm
%matplotlib inline

# Get the dataset

In [0]:
# Colab's file access feature
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
# json_file = '/content/gdrive/My Drive/kaggle/kaggle.json'

In [0]:
!mkdir .kaggle
cd kaggle\ 

In [0]:
# To get the API token from kaggle (kaggle.json file)
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

In [0]:
#download data, will take 30-60 seconds
!kaggle competitions download -c ammi-2020-convnets

In [0]:
#unzip all data for usage, will take few minutes
!unzip -qq ammi-2020-convnets.zip

#USING FASTAI

In [0]:
# using defult transforms
tfms = get_transforms(**transform_kwargs)

# get the dataset
data = (ImageList.from_folder('/train') #Where to find the data? -> in path and its subfolders
        .split_by_rand_pct()              #How to split in train/valid? -> use the folders
        .label_from_folder()            #How to label? -> depending on the folder of the filenames
        .transform(tfms, size=224)       #Data augmentation? -> use tfms with a size of 64
        .add_test_folder('/test')
        .databunch(bs = 64)) 
data.classes

In [0]:
# specify the model you wanna use
base_arch = lambda arg: pm.se_resnet50(num_classes=5, pretrained=None)
learner = vision.cnn_learner(data, 
                             base_arch = base_arch, 
                             pretrained = False, 
                             metrics = vision.accuracy)


In [0]:
# Train the model
learner.fit_one_cycle(3) # you can specify the lr if you want o.w will the deafult one

# to save the model
learner.save('model_1')

In [0]:
# Get the prediction

predicted_probs, _ = learner.TTA(ds_type = fastai.basic_data.DatasetType.Test)
predicted_probs, y = learner.get_preds(ds_type = fastai.basic_data.DatasetType.Test)

_, predicted_classes = predicted_probs.max(dim=1)
class_labels = np.array(['cbb','cbsd','cgm','cmd','healthy'])
predicted_class_labels = class_labels[predicted_classes]

filenames = np.array([item.name for item in data.test_ds.items])
submission = (pd.DataFrame.from_dict({'Category': predicted_class_labels,'Id': filenames}))
sub_file = '/content/gdrive/My Drive/Colab Notebooks/Kaggle/cassava_challenge/late_submission_pseudo_start_8.csv'
submission.to_csv(sub_file, header=True, index=False)

# USIN PYTORCH

```
# This is formatted as code
```



In [0]:
data_path = "train/train"
test_path = "test/test"
extraimage_path = "extraimages"

In [0]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([ transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor()])

normalize = transforms.Normalize(mean=mean, std=std)

In [0]:
class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, 224, 224), classCategory

In [0]:
train_data = CassavaDataset(data_path, transform=train_transforms)
test_data = CassavaDataset(test_path, transform=test_transforms)

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

In [0]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [0]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
# Define Models 

def se_resnext50_32x4d(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.se_resnext50_32x4d(pretrained=pretrained)
    return model

model = se_resnext50_32x4d(pretrained=False)

In [0]:
for param in model.parameters():
    param.requires_grad = False

# new final layer with 5 classes
model.avg_pool = torch.nn.AdaptiveAvgPool2d(1)
num_ftrs = model.last_linear.in_features
model.last_linear = torch.nn.Linear(num_ftrs, 5)
if use_gpu:
    model = model.cuda()

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [0]:
def train(model, criterion, data_loader, optimizer, num_epochs):
    """Simple training loop for a PyTorch model.""" 
    
    # Make sure model is in training mode.
    model.train()
    
    # Move model to the device (CPU or GPU).
    model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None
    
    # Loop over epochs.
    for epoch in range(num_epochs):
        
      # Loop over data.
      for batch_idx, (data, target) in enumerate(data_loader):
            
          # Forward pass.
          output = model(data.to(device))
          loss = criterion(output.to(device), target.to(device))
          
          # Backward pass.
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          # NOTE: It is important to call .item() on the loss before summing.
          if ema_loss is None:
            ema_loss = loss.item()
          else:
            ema_loss += (loss.item() - ema_loss) * 0.01 
          
      # Print out progress the end of epoch.
      print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, ema_loss),)
  

In [0]:
def test(model, data_loader):
    """Measures the accuracy of a model on a data set.""" 
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0

    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        
        # Loop over test data.
        for data, target in data_loader:
          
            # Forward pass.
            output = model(data.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count number of correct predictions.
            correct += pred.cpu().eq(target.view_as(pred)).sum().item()

    # Print test accuracy.
    percent = 100. * correct / len(data_loader.dataset)
    print(f'Accuracy: {correct} / {len(data_loader.dataset)} ({percent:.0f}%)')
    torch.save(model.state_dict(), 'model.ckpt')
    return percent

In [0]:
train(model, criterion, train_loader, optimizer, num_epochs=1)
test(model, test_loader)

In [0]:
# load saved model to make predictions on test data
model.load_state_dict(torch.load('model.ckpt'))
model.eval()

In [0]:
## Make Submission